In [1]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-12.2  games	       include	lib64	   man	 share
colab  cuda-12	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [2]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [3]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Sat Jun  8 22:03:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Write a cu file contain the host and kernel code
%%writefile coordinate.cu

#include <cuda_runtime.h>
#include <stdio.h>

__global__ void printCoordinate(int *A,const int nx,const int ny)
{
  int ix=threadIdx.x+blockIdx.x*blockDim.x;
  int iy=threadIdx.y+blockIdx.y*blockDim.y;
  unsigned int idx=iy*nx+ix;
  printf("thread_id(%d,%d) block_id(%d,%d) coordinate(%d,%d)"
          "global index %2d ival %2d\n",threadIdx.x,threadIdx.y,
          blockIdx.x,blockIdx.y,ix,iy,idx,A[idx]);
}

int main(int argc,char ** argv)
{
  cudaSetDevice(0);
  int nx = 8;
  int ny = 6;
  // Malloc
  int* A_host = (int*)malloc(nx*ny*sizeof(int));
  for(int i = 0; i<nx*ny; ++i){
      A_host[i] = 0;
  }

  // cudaMalloc
  int *A_dev=NULL;
  cudaMalloc((void**)&A_dev, nx*ny*sizeof(int));
  cudaMemcpy(A_dev, A_host, nx*ny*sizeof(int), cudaMemcpyHostToDevice);

  dim3 block(4,2);
  dim3 grid((nx-1)/block.x+1,(ny-1)/block.y+1);

  printCoordinate<<<grid,block>>>(A_dev,nx,ny);
  cudaDeviceSynchronize();

  // Free
  cudaFree(A_dev);
  free(A_host);

  cudaDeviceReset();
  return 0;
}

Writing coordinate.cu


In [5]:
!nvcc --list-gpu-arch

compute_50
compute_52
compute_53
compute_60
compute_61
compute_62
compute_70
compute_72
compute_75
compute_80
compute_86
compute_87
compute_89
compute_90


In [6]:
# Compile the code. The flag is needed if you use the Tesla K80.
#!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 coordinate.cu -o coordinate
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 coordinate.cu -o coordinate

In [7]:
!./coordinate

thread_id(0,0) block_id(1,0) coordinate(4,0)global index  4 ival  0
thread_id(1,0) block_id(1,0) coordinate(5,0)global index  5 ival  0
thread_id(2,0) block_id(1,0) coordinate(6,0)global index  6 ival  0
thread_id(3,0) block_id(1,0) coordinate(7,0)global index  7 ival  0
thread_id(0,1) block_id(1,0) coordinate(4,1)global index 12 ival  0
thread_id(1,1) block_id(1,0) coordinate(5,1)global index 13 ival  0
thread_id(2,1) block_id(1,0) coordinate(6,1)global index 14 ival  0
thread_id(3,1) block_id(1,0) coordinate(7,1)global index 15 ival  0
thread_id(0,0) block_id(0,2) coordinate(0,4)global index 32 ival  0
thread_id(1,0) block_id(0,2) coordinate(1,4)global index 33 ival  0
thread_id(2,0) block_id(0,2) coordinate(2,4)global index 34 ival  0
thread_id(3,0) block_id(0,2) coordinate(3,4)global index 35 ival  0
thread_id(0,1) block_id(0,2) coordinate(0,5)global index 40 ival  0
thread_id(1,1) block_id(0,2) coordinate(1,5)global index 41 ival  0
thread_id(2,1) block_id(0,2) coordinate(2,5)glob